In [16]:
import pandas as pd
import os
import acquire_nick as get
import prepare_nick as prep

In [69]:
df = get.acquire_data()

Using cached CSV


In [4]:
df.dtypes

Unnamed: 0                int64
cast                     object
crew                     object
id                        int64
title                    object
genres                   object
budget                    int64
overview                 object
popularity              float64
production_companies     object
production_countries     object
revenue                 float64
runtime                 float64
vote_average            float64
vote_count              float64
keywords                 object
release_date             object
dtype: object

In [70]:
# Deleted columns Unnamed: 0
df = df.drop(['Unnamed: 0'], axis=1)


In [14]:
df.head(1)

,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,TomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette,JohnLasseter JossWhedon AndrewStanton JoelCohen AlecSokolow BonnieArnold EdCatmull RalphGuggenheim SteveJobs LeeUnkrich RalphEggleston RobertGordon MaryHelenLeasman KimBlanchette MarilynMcCoppen RandyNewman DaleE.Grahn RobinCooper JohnLasseter PeteDocter JoeRanft PatsyBouge NormDeCarlo AshBrannon RandyNewman RomanFigun DonDavis JamesFlamberg MaryBethSmith RickMackay SusanBradley WilliamReeves RandyNewman AndrewStanton PeteDocter GaryRydstrom KarenRobertJackson ChrisMontan RichQuade MichaelBerenstein ColinBrady DaveyCrockettFeiten AngieGlocka RexGrignon TomK.Gurney JimmyHayward HalT.Hickel KarenKiser AnthonyB.LaMolinara GuionneLeroy BudLuckey LesMajor GlennMcQueen MarkOftedal JeffPidgeon JeffPratt SteveRabatich RogerRose SteveSegal DougSheppeck AlanSperling DougSweetland DavidTart KenWillard ThomasPorter MarkThomasHenne OrenJacob DarwynPeachey MitchPrater BrianM.Rosen SharonCalahan GalynSusman WilliamCone ShelleyDanielsLekven BobPauley BudLuckey AndrewStanton WilliamCone SteveJohnson DanHaskett TomHolloway JeanGillmore DesiréeMourad KellyO'Connell SonokoKonishi AnnM.Rockwell JulieM.McDonald RobinLee TomFreeman AdaCochavi DanaMulligan DeirdreMorrison LoriLombardo EllenDevine LaurenBethStrogoff GaryRydstrom GarySummers TimHolland PatJackson TomMyers J.R.Grubbs SusanSanford SusanPopovic DanEngstrom RuthLambert MickieMcGowan,862,Toy Story,Animation Comedy Family,30000000,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,PixarAnimationStudios,UnitedStatesofAmerica,373554033.0,81.0,7.7,5415.0,jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life,1995-10-30


Plans:
Extract the number of names inside of crew and create a column that contains a count of names that appear. Consider the same for cast.     

Scaling will be needed for the differences in scope among votes, popularity, budget, revenue, et cetera.

I'm curious as to whether or not number of keywords, or the types of keywords themselves will have a signifcant influence on the predictability of scores or budget. Presumably the film studio will have that effect, and the production company may also. 

ID should probably be cast as a category, or at the very least a generic string, since there is no rank or meaning of the order per id. 

Release date could be converted to a pandas date-time format for the sake of time-series analysis. Otherwise, it seems much of what we will be doing going forward can be framed under many different methodologies. 

In [17]:
prep.describe_data(df)

This dataframe has 7457 rows and 16 columns.

--------------------------------------
--------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7457 entries, 0 to 7456
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cast                  7423 non-null   object 
 1   crew                  7444 non-null   object 
 2   id                    7457 non-null   int64  
 3   title                 7457 non-null   object 
 4   genres                7434 non-null   object 
 5   budget                7457 non-null   int64  
 6   overview              7435 non-null   object 
 7   popularity            7457 non-null   float64
 8   production_companies  7039 non-null   object 
 9   production_countries  7301 non-null   object 
 10  revenue               7457 non-null   float64
 11  runtime               7451 non-null   float64
 12  vote_average          7457 non-null   float64
 13 

TypeError: '<=' not supported between instances of 'str' and 'float'

Because the total proportion of the data frame affected by nulls is barely one percent, and since the columns containing these values can not be inferred due to their qualitative nature that varies from film to film, dropping them from the data frame altogether is perfectly acceptable. 

In [71]:
df.shape
# first lets see what happens when I remove na's from just the keywords column; ideally, most of the 
df2 = df.copy()
# I will use this at the end for comparison's sake. I will remove all the nas from this data frame at once

In [33]:
df = df.dropna(subset=['keywords'])

In [34]:
prep.nulls(df)

Null Values by Column: 
                      count  percentage
production_companies    236        3.48
production_countries     68        1.00
cast                     17        0.25
overview                 12        0.18
runtime                   4        0.06
crew                      3        0.04
genres                    3        0.04
id                        0        0.00
title                     0        0.00
budget                    0        0.00
popularity                0        0.00
revenue                   0        0.00
vote_average              0        0.00
vote_count                0        0.00
keywords                  0        0.00
release_date              0        0.00
 
 Total Number of Missing Values: 343 

 Proportion of Nulls in Dataframe: 0.0032

--------------------------------------
--------------------------------------
Row-by-Row Nulls
                 percent_cols_missing  num_rows
num_cols_missing                               
1                    

In [35]:
# now with production companies
df = df.dropna(subset=['production_companies'])

In [36]:
prep.nulls(df)

Null Values by Column: 
                      count  percentage
production_countries     19        0.29
overview                 11        0.17
cast                      5        0.08
runtime                   4        0.06
crew                      2        0.03
genres                    1        0.02
id                        0        0.00
title                     0        0.00
budget                    0        0.00
popularity                0        0.00
production_companies      0        0.00
revenue                   0        0.00
vote_average              0        0.00
vote_count                0        0.00
keywords                  0        0.00
release_date              0        0.00
 
 Total Number of Missing Values: 42 

 Proportion of Nulls in Dataframe: 0.0004

--------------------------------------
--------------------------------------
Row-by-Row Nulls
                 percent_cols_missing  num_rows
num_cols_missing                               
1                     

In [37]:
df.shape

(6538, 16)

In [72]:
# now I will drop the rest. 
df = df.dropna()

In [39]:
df.shape

(6502, 16)

In [40]:
df2 = df2.dropna()

In [41]:
df2.shape

(6502, 16)

In [ ]:
# the same result. 

In [47]:
# check for duplicates:
df['duplicate'] = df.duplicated()

In [52]:
dups = df[df['duplicate'] == True]
# at this point I am reminded of the films where budgets are zero. 
# these indicate reissues or re-releases, and in the event we label a film as being successful
# on the basis of revenue == 2x budget, for example, these have an unfair, inaccurate advantage.
# I am removing all films where the budget is equal to zero. Then I will check duplicates again. 
dups.count()
# something is definitely weird about this. 

cast                    53
crew                    53
id                      53
title                   53
genres                  53
budget                  53
overview                53
popularity              53
production_companies    53
production_countries    53
revenue                 53
runtime                 53
vote_average            53
vote_count              53
keywords                53
release_date            53
duplicate               53
dtype: int64

In [74]:
# regardless, I will keep first, where the budget is not zero, and drop the rest. one way to do that 
# is to first resort the dataframe. 
df = df.sort_values(by='budget', ascending=False, na_position='last')
# keep first instance of the duplicate
df.duplicated(keep='first')

5013    False
3972    False
6232    False
3745    False
5270    False
        ...  
953     False
4451    False
4450    False
4449    False
7451    False
Length: 6502, dtype: bool

In [75]:
# in case something goes wrong 
df_safe = df.copy()
# change inplace to True if the results are good, for the prepare function
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)

In [76]:
df.shape
# dupl

(6449, 16)

In [60]:
df[df['budget'] == 0].count()

cast                    1557
crew                    1557
id                      1557
title                   1557
genres                  1557
budget                  1557
overview                1557
popularity              1557
production_companies    1557
production_countries    1557
revenue                 1557
runtime                 1557
vote_average            1557
vote_count              1557
keywords                1557
release_date            1557
duplicate               1557
dtype: int64

Hold the phone. That's way more than I thought. We may need to actually handle this based on imputation. 

In [62]:
df.shape

(6502, 17)